In [ ]:
!nvidia-smi

Wed May  1 20:49:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!mkdir -p /data/sets/nuscenes  # Make the directory to store the nuScenes dataset in.

!wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.

!tar -xf v1.0-mini.tgz -C /data/sets/nuscenes  # Uncompress the nuScenes mini split.

!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.

--2024-05-01 21:38:30--  https://www.nuscenes.org/data/v1.0-mini.tgz
Resolving www.nuscenes.org (www.nuscenes.org)... 18.64.174.12, 18.64.174.36, 18.64.174.111, ...
Connecting to www.nuscenes.org (www.nuscenes.org)|18.64.174.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4167696325 (3.9G) [application/x-tar]
Saving to: ‘v1.0-mini.tgz.1’

v1.0-mini.tgz.1     100%[===================>]   3.88G  79.3MB/s    in 42s     

2024-05-01 21:39:13 (94.4 MB/s) - ‘v1.0-mini.tgz.1’ saved [4167696325/4167696325]



In [ ]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='/data/sets/nuscenes', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.851 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [ ]:
cat = []
for i in range(len(nusc.category)):
    print(nusc.category[i]['name'])
    cat.append(nusc.category[i]['name'])

human.pedestrian.adult
human.pedestrian.child
human.pedestrian.wheelchair
human.pedestrian.stroller
human.pedestrian.personal_mobility
human.pedestrian.police_officer
human.pedestrian.construction_worker
animal
vehicle.car
vehicle.motorcycle
vehicle.bicycle
vehicle.bus.bendy
vehicle.bus.rigid
vehicle.truck
vehicle.construction
vehicle.emergency.ambulance
vehicle.emergency.police
vehicle.trailer
movable_object.barrier
movable_object.trafficcone
movable_object.pushable_pullable
movable_object.debris
static_object.bicycle_rack


In [ ]:
classes = ['human.pedestrian.adult', 'human.pedestrian.child','human.pedestrian.police_officer','human.pedestrian.construction_worker','vehicle.car','vehicle.bicycle']
pedestrians = ['human.pedestrian.adult', 'human.pedestrian.child','human.pedestrian.police_officer','human.pedestrian.construction_worker']

In [ ]:
print('Total number of samples')
print(len(nusc.sample))

total_no_of_samples = len(nusc.sample)

#print('Total number of images')
#print(len(nusc.sample*6)) #6 different cameras

Total number of samples
404


In [ ]:
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import Box
import numpy as np
from nuscenes.utils.geometry_utils import points_in_box,BoxVisibility,box_in_image
def get_sample_data(nusc_object, sample_data_token, box_vis_level=BoxVisibility.ANY, selected_anntokens=None):
    """
    Returns the data path as well as all annotations related to that sample_data(single image).
    Note that the boxes are transformed into the current sensor's coordinate frame.
    :param sample_data_token: <str>. Sample_data token(image token).
    :param box_vis_level: <BoxVisibility>. If sample_data is an image, this sets required visibility for boxes.
    :param selected_anntokens: [<str>]. If provided only return the selected annotation.
    :return: (data_path <str>, boxes [<Box>], camera_intrinsic <np.array: 3, 3>)
    """

    # Retrieve sensor & pose records
    sd_record = nusc_object.get('sample_data', sample_data_token)
    cs_record = nusc_object.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
    sensor_record = nusc_object.get('sensor', cs_record['sensor_token'])
    pose_record = nusc_object.get('ego_pose', sd_record['ego_pose_token'])

    sample_record = nusc_object.get('sample',sd_record['sample_token'])
    data_path = nusc_object.get_sample_data_path(sample_data_token)

    if sensor_record['modality'] == 'camera':
        cam_intrinsic = np.array(cs_record['camera_intrinsic'])
        imsize = (sd_record['width'], sd_record['height'])
    else:
        cam_intrinsic = None
        imsize = None

    # Retrieve all sample annotations and map to sensor coordinate system.
    if selected_anntokens is not None:
        boxes = list(map(nusc_object.get_box, selected_anntokens))
    else:
        boxes = nusc_object.get_boxes(sample_data_token)
        selected_anntokens = sample_record['anns']

    # Make list of Box objects including coord system transforms.
    box_list = []
    ann_list = []
    for box,ann in zip(boxes,selected_anntokens):

        # Move box to ego vehicle coord system
        box.translate(-np.array(pose_record['translation']))
        box.rotate(Quaternion(pose_record['rotation']).inverse)

        #  Move box to sensor coord system
        box.translate(-np.array(cs_record['translation']))
        box.rotate(Quaternion(cs_record['rotation']).inverse)

        if sensor_record['modality'] == 'camera' and not \
                box_in_image(box, cam_intrinsic, imsize, vis_level=box_vis_level):
            continue

        box_list.append(box)
        ann_list.append(ann)
    #this is for a single sample image
    return data_path, box_list, ann_list, cam_intrinsic #single image info

In [ ]:
def threeD_2_twoD(boxsy,intrinsic): #input is a single annotation box
    '''
    given annotation boxes and intrinsic camera matrix
    outputs the 2d bounding box coordinates as a list (all annotations for a particular sample image)
    '''
    corners = boxsy.corners()
    x = corners[0,:]
    y = corners[1,:]
    z = corners[2,:]
    x_y_z = np.array((x,y,z))
    orthographic = np.dot(intrinsic,x_y_z)
    perspective_x = orthographic[0]/orthographic[2]
    perspective_y = orthographic[1]/orthographic[2]
    perspective_z = orthographic[2]/orthographic[2]

    min_x = np.min(perspective_x)
    max_x = np.max(perspective_x)
    min_y = np.min(perspective_y)
    max_y = np.max(perspective_y)



    return min_x,max_x,min_y,max_y



def all_3d_to_2d(boxes,anns,intrinsic): #input 3d boxes, annotation key lists, intrinsic matrix (one image)
    x_min=[]
    x_max=[]
    y_min=[]
    y_max =[]
    width=[]
    height=[]
    objects_detected =[]
    orig_objects_detected =[]


    for j in range(len(boxes)): #iterate through boxes
        box=boxes[j]

        if box.name in classes: #if the box.name is in the classes we want to detect

            if box.name in pedestrians:
                orig_objects_detected.append("pedestrian")
            elif box.name == "vehicle.car":
                orig_objects_detected.append("car")
            else:
                orig_objects_detected.append("cyclist")
            #print(box)

            visibility = nusc.get('sample_annotation', '%s' %anns[j])['visibility_token'] #give annotation key
            visibility = int(visibility)


            if visibility > 1: #more than 40% visible in the panoramic view of the the cameras


                center = box.center #get boxe's center

                center = np.dot(intrinsic,center)
                center_point = center/(center[2]) #convert center point into image plane




                if center_point[0] <-100 or center_point[0] > 1700 or center_point[1] <-100 or center_point[1] >1000:
                    #if center of bounding box is outside of the image, do not annotate
                    pass

                else:
                    min_x, max_x, min_y, max_y = threeD_2_twoD(box,intrinsic) #converts box into image plane
                    w = max_x - min_x
                    h = max_y - min_y


                    x_min.append(min_x)
                    x_max.append(max_x)
                    y_min.append(min_y)
                    y_max.append(max_y)
                    width.append(w)
                    height.append(h)
                    if box.name in pedestrians:
                        objects_detected.append("pedestrian")
                    elif box.name == "vehicle.car":
                        objects_detected.append("car")
                    else:
                        objects_detected.append("cyclist")


            else:
                pass

    return x_min,x_max,y_min,y_max,width,height,objects_detected,orig_objects_detected #for a single image

In [ ]:
def extract_bounding_box(i,camera_name): #give a single sample number and camera name

    '''
    input sample number i, camera name
    outputs min x, max x, min y max y, width and height of bounding box in image coordinates
    2d bounding box
    options for camera name : CAM_FRONT, CAM_FRONT_RIGHT, CAM_FRONT_LEFT, CAM_BACK, CAM_BACK_RIGHT,CAM_BACK_LEFT
    '''

    nusc.sample[i] #one image

    camera_token = nusc.sample[i]['data']['%s' %camera_name] #one camera, get the camera token

    path, boxes, anns, intrinsic_matrix = get_sample_data(nusc,'%s' %camera_token) #gets data for one image

    x_min, x_max,y_min,y_max,width,height, objects_detected,orig_objects_detected = all_3d_to_2d(boxes,anns, intrinsic_matrix)

    return x_min, x_max, y_min, y_max, width, height, path, boxes,intrinsic_matrix, objects_detected,orig_objects_detected
    #info for a single image

In [ ]:
import os.path
def create_annotation_directory(camera):
    current_dir =os.getcwd()
    #current_dir ="%s/annotation" %pwd
    dirName ="%s/annotation/%s_anno" %(current_dir,camera)
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        print("Directory " , dirName ,  " Created ")
    else:
        print("Directory " , dirName ,  " already exists")

In [ ]:
from lxml import etree as ET
def write_xml_annotation(x_min,x_max,y_min,y_max,width,height,path,boxes,objects_detected): #single image info
    #detected_items =[]
    #import xml.etree.cElementTree as ET
    path_split = path.split("/")
    full_image_name = path_split[-1]
    name =full_image_name.split(".")[0]

    root = ET.Element("annotation")


    ET.SubElement(root, "folder").text = "%s" %camera
    ET.SubElement(root, "filename").text = "%s" %full_image_name
    ET.SubElement(root, "path").text = "%s" %path

    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "nuTonomy-nuscenes"

    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text="1600"
    ET.SubElement(size,"height").text="900"
    ET.SubElement(size,"depth").text="3"
    ET.SubElement(root, "segmented").text = "0"

    for j in range(len(objects_detected)): #

        ob= ET.SubElement(root, "object")
        ET.SubElement(ob,"name").text="%s" %objects_detected[j]
        ET.SubElement(ob,"pose").text="Unspecified"
        ET.SubElement(ob, "truncated").text="truncated"
        ET.SubElement(ob, "difficult").text="0"

        bb = ET.SubElement(ob,"bndbox")
        ET.SubElement(bb,"xmin").text="%s" %x_min[j]
        ET.SubElement(bb,"ymin").text="%s" %y_min[j]
        ET.SubElement(bb,"xmax").text="%s" %x_max[j]
        ET.SubElement(bb,"ymax").text="%s" %y_max[j]


    filename = "%s/%s.xml" %(dirName,name)
    tree = ET.ElementTree(root)
    #tree.write("%s/%s.xml" %(dirName,name),pretty_print=True)
    tree.write("%s" %filename, pretty_print=True)

    return filename #file a single file

In [ ]:
print('Total number of samples')
print(len(nusc.sample))

total_no_of_samples = len(nusc.sample)


Total number of samples
404


In [ ]:
camera_names =['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT', 'CAM_BACK_LEFT']
i = 0
detected_items =[]
orig_detected_items=[]
obs = []

file=[]

for camera in camera_names: #iterate through all cameras
    print(camera)
    create_annotation_directory(camera)
    current_dir =os.getcwd()
    dirName ="%s/annotation/%s_anno" %(current_dir,camera) #current directory's name
    #we are looking at one camera now
    for sample_number in range(total_no_of_samples):#look at a single image
        x_min, x_max,y_min,y_max,width,height, path, boxes, intrinsic_matrix,objects_detected,orig_objects_detected = extract_bounding_box(sample_number, '%s' %camera)
        write_xml_annotation(x_min,x_max,y_min,y_max,width,height,path,boxes,objects_detected)





CAM_FRONT
Directory  /content/annotation/CAM_FRONT_anno  Created 
CAM_FRONT_RIGHT
Directory  /content/annotation/CAM_FRONT_RIGHT_anno  Created 
CAM_FRONT_LEFT
Directory  /content/annotation/CAM_FRONT_LEFT_anno  Created 
CAM_BACK
Directory  /content/annotation/CAM_BACK_anno  Created 
CAM_BACK_RIGHT
Directory  /content/annotation/CAM_BACK_RIGHT_anno  Created 
CAM_BACK_LEFT
Directory  /content/annotation/CAM_BACK_LEFT_anno  Created 


In [ ]:
import os.path
from os import listdir
import xml.etree.ElementTree as ET
camera_names =['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_FRONT_LEFT', 'CAM_BACK', 'CAM_BACK_RIGHT', 'CAM_BACK_LEFT']

def list_of_files(camera):
    current_dir =os.getcwd()
    #current_dir ="%s/annotation" %pwd

    dirName ="%s/annotation/%s_anno" %(current_dir,camera)
    files = os.listdir(dirName)

    return files, dirName, current_dir


In [ ]:
with open("file_list.txt", "w") as file_list:

    for camera in camera_names:

        files, dirName, current_dir = list_of_files(camera)
        for f in files:
            file_list.write('%s/%s' %(dirName,f))
            file_list.write('\n')


In [ ]:
file = open("file_list.txt", "r")
total_files = []
cam =[]
directory=[]
#file_number =[]
#i = 0

for line in file:


    total_files.append(line.strip())

    #file_number.append(i)
    #i = i + 1

In [ ]:
all_files = total_files

In [ ]:
len(all_files)

2424

In [ ]:
import random
random.seed(8)


shuffled_list = random.sample(all_files, len(all_files))

In [ ]:
random.seed(8)
shuffled_list_train = random.sample(shuffled_list,int(0.7*len(shuffled_list)))

In [ ]:
remaining_list = [x for x in shuffled_list if x not in shuffled_list_train ]

In [ ]:
random.seed(8)
shuffled_list_val = random.sample(remaining_list,int(0.5*len(remaining_list)))

In [ ]:
shuffled_list_test = [x for x in remaining_list if x not in shuffled_list_val ]

In [ ]:
(len(shuffled_list_val)/len(shuffled_list)) * 100

15.016501650165019

In [ ]:
shuffled_list_val[0]

'/content/annotation/CAM_BACK_LEFT_anno/n008-2018-08-01-15-16-36-0400__CAM_BACK_LEFT__1533151621447405.xml'

In [ ]:
shuffled_list_val[2]

'/content/annotation/CAM_FRONT_LEFT_anno/n015-2018-11-21-19-38-26+0800__CAM_FRONT_LEFT__1542800371904844.xml'

In [ ]:
import os.path
item = ['train', 'test', 'val']
todo = ['annotations', 'images']

def create_directory(item, todo):
    current_dir =os.getcwd()
    #current_dir ="%s/annotation" %pwd
    dirName ="%s/nu_%s_%s/" %(current_dir,item, todo)
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        print("Directory " , dirName ,  " Created ")
    else:
        print("Directory " , dirName ,  " already exists")

In [ ]:
for i in item:
    for j in todo:
        create_directory(i,j)

Directory  /content/nu_train_annotations/  Created 
Directory  /content/nu_train_images/  Created 
Directory  /content/nu_test_annotations/  Created 
Directory  /content/nu_test_images/  Created 
Directory  /content/nu_val_annotations/  Created 
Directory  /content/nu_val_images/  Created 


In [ ]:
import shutil
current_dir =os.getcwd()
dirName = 'nu_test_annotations'
for files in shuffled_list_test:
    #f = '%s/%s' %(files[1],files[0])
    #print(files.strip())

    shutil.copy2(files.strip(),'%s/%s' %(current_dir,dirName))

In [ ]:
import os
import shutil

current_dir = os.getcwd()
dirName = 'nu_test_images'

# Create the destination directory if it doesn't exist
if not os.path.exists(dirName):
    os.makedirs(dirName)

for files in shuffled_list_test:
    try:
        f = files.split('/')
        camera_name = f[-2].split('_anno')[0]

        source_file_name = f[-1].split('xml')[0] + '.jpg'
        source_path = os.path.join(current_dir, 'data', 'nuscenes', 'samples', camera_name, source_file_name)

        destination_path = os.path.join(current_dir, dirName, source_file_name)

        print(f"Copying file from '{source_path}' to '{destination_path}'")

        shutil.copy2(source_path, destination_path)
        print(f"File '{source_file_name}' copied successfully!")
    except FileNotFoundError as e:
        print(f"Error: File '{source_file_name}' not found at '{source_path}'. Skipping...")
    except Exception as e:
        print(f"An error occurred: {e}")

print("Copying process completed.")


Copying file from '/content/data/nuscenes/samples/CAM_BACK/n008-2018-08-01-15-16-36-0400__CAM_BACK__1533151616937558..jpg' to '/content/nu_test_images/n008-2018-08-01-15-16-36-0400__CAM_BACK__1533151616937558..jpg'
Error: File 'n008-2018-08-01-15-16-36-0400__CAM_BACK__1533151616937558..jpg' not found at '/content/data/nuscenes/samples/CAM_BACK/n008-2018-08-01-15-16-36-0400__CAM_BACK__1533151616937558..jpg'. Skipping...
Copying file from '/content/data/nuscenes/samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127612404..jpg' to '/content/nu_test_images/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127612404..jpg'
Error: File 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127612404..jpg' not found at '/content/data/nuscenes/samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127612404..jpg'. Skipping...
Copying file from '/content/data/nuscenes/samples/CAM_FRONT_RIGHT/n015-2018-10-08-15-36-50+0800__CAM_FRONT_RIGHT__1538984250920339..jpg' to '/conten

In [ ]:
b = shuffled_list_train[0].split('/')
print(b)


['', 'content', 'annotation', 'CAM_BACK_anno', 'n008-2018-08-30-15-16-55-0400__CAM_BACK__1535657114637558.xml']


In [ ]:
print(b[-1].split('xml')[0])

n008-2018-08-30-15-16-55-0400__CAM_BACK__1535657114637558.
